## Create Training data for Mapping between Wikipedia2vec (wordpiece) and BERT embeddings

In [1]:
import json
import torch
import torch.nn as nn

import logging
import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaForMaskedLM

In [2]:
embeddings_dict = {}
with open("/data_ssds/disk11/slinzbach/enwiki_20180420_500d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[-500:], "float32")
        embeddings_dict[token] = vector

In [3]:
len(embeddings_dict)

4529833

In [4]:
entities = {}
wordpiece = {}

for key in embeddings_dict.keys():
    
    if "ENTITY/" in key:
        entities[key] = embeddings_dict[key]
        
    else:
        wordpiece[key] = embeddings_dict[key]

In [ ]:
from datasets import Dataset

ds = Dataset.from_dict({"Entity": [1]*len(wordpiece), 
                        "label": list(wordpiece.keys()),
                        "embeddings": list(wordpiece.values()) })
ds[0]

In [ ]:
ds.add_faiss_index(column='embeddings')

In [12]:
ds.save_to_disk('/data_ssds/disk11/slinzbach/words_faiss')

Saving the dataset (0/8 shards):   0%|          | 0/1937423 [00:00<?, ? examples/s]

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
entities_vecs = np.stack([v for v in entities.values()])

In [5]:
len(entities)

2592410

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [11]:
len(tokenizer.get_vocab().keys())

50265

In [13]:
letters = set.union(*[set(k) for k in tokenizer.get_vocab()])
letters

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '¡',
 '¢',
 '£',
 '¤',
 '¥',
 '¦',
 '§',
 '¨',
 '©',
 'ª',
 '«',
 '¬',
 '®',
 '¯',
 '°',
 '±',
 '²',
 '³',
 '´',
 'µ',
 '¶',
 '·',
 '¸',
 '¹',
 'º',
 '»',
 '¼',
 '½',
 '¾',
 '¿',
 'À',
 'Á',
 'Â',
 'Ã',
 'Ä',
 'Å',
 'Æ',
 'Ç',
 'È',
 'É',
 'Ê',
 'Ë',
 'Ì',
 'Í',
 'Î',
 'Ï',
 'Ð',
 'Ñ',
 'Ò',
 'Ó',
 'Ô',
 'Õ',
 'Ö',
 '×',
 'Ø',
 'Ù',
 'Ú',
 'Û',
 'Ü',
 'Ý',
 'Þ',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ê

In [6]:
common_vocab = []
for tok in tokenizer.get_vocab().keys():
    tok = tok.replace('Ġ', '')
    if "ENTITY/"+tok in entities:
        common_vocab.append("ENTITY/"+tok)
    elif tok.lower() in wordpiece:
        common_vocab.append(tok)
    else:
        common_vocab.append(None)

In [7]:
bert = RobertaForMaskedLM.from_pretrained('roberta-base')

In [56]:
bert.roberta.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(50265, 768, padding_idx=1)
  (position_embeddings): Embedding(514, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [36]:
decoder_lm2wiki = nn.Linear(500, 768, bias = False)

In [48]:
decoder_wiki2lm = nn.Linear(768, 500, bias = False)
decoder_wiki2lm.weight = torch.nn.Parameter(decoder_lm2wiki.weight.T)

In [49]:
decoder_wiki2lm.weight

Parameter containing:
tensor([[ 0.0154, -0.0111,  0.0193,  ..., -0.0035,  0.0346, -0.0404],
        [-0.0065,  0.0133,  0.0286,  ...,  0.0333, -0.0361, -0.0017],
        [ 0.0084,  0.0123,  0.0040,  ..., -0.0288,  0.0225,  0.0309],
        ...,
        [ 0.0194,  0.0085, -0.0365,  ...,  0.0398, -0.0375,  0.0289],
        [ 0.0301,  0.0250, -0.0440,  ..., -0.0376, -0.0370,  0.0403],
        [-0.0269,  0.0027, -0.0163,  ..., -0.0293, -0.0301,  0.0427]],
       requires_grad=True)

In [50]:

with torch.no_grad():
    decoder_lm2wiki.weight[:1,:1] = 0

In [52]:
decoder_lm2wiki.weight

Parameter containing:
tensor([[ 0.0000, -0.0065,  0.0084,  ...,  0.0194,  0.0301, -0.0269],
        [-0.0111,  0.0133,  0.0123,  ...,  0.0085,  0.0250,  0.0027],
        [ 0.0193,  0.0286,  0.0040,  ..., -0.0365, -0.0440, -0.0163],
        ...,
        [-0.0035,  0.0333, -0.0288,  ...,  0.0398, -0.0376, -0.0293],
        [ 0.0346, -0.0361,  0.0225,  ..., -0.0375, -0.0370, -0.0301],
        [-0.0404, -0.0017,  0.0309,  ...,  0.0289,  0.0403,  0.0427]],
       requires_grad=True)

In [53]:
decoder_wiki2lm.weight

Parameter containing:
tensor([[ 0.0000, -0.0111,  0.0193,  ..., -0.0035,  0.0346, -0.0404],
        [-0.0065,  0.0133,  0.0286,  ...,  0.0333, -0.0361, -0.0017],
        [ 0.0084,  0.0123,  0.0040,  ..., -0.0288,  0.0225,  0.0309],
        ...,
        [ 0.0194,  0.0085, -0.0365,  ...,  0.0398, -0.0375,  0.0289],
        [ 0.0301,  0.0250, -0.0440,  ..., -0.0376, -0.0370,  0.0403],
        [-0.0269,  0.0027, -0.0163,  ..., -0.0293, -0.0301,  0.0427]],
       requires_grad=True)

In [9]:
i_embeddings = None
for p in bert.roberta.embeddings.word_embeddings.parameters():
    i_embeddings = p
    break


In [10]:
i_embeddings.size()

torch.Size([50265, 768])

In [16]:
common_vocab[8909:]

['MD',
 'immigrant',
 'ENTITY/Construction',
 'Born',
 None,
 'Wade',
 'visa',
 'genuine',
 'electronics',
 'Sat',
 'sponsors',
 'ENTITY/Montana',
 'spell',
 'ENTITY/Sachs',
 'Et',
 'foster',
 'locker',
 'explaining',
 'Age',
 'gunman',
 'sauce',
 'cry',
 'stimulus',
 'array',
 'compare',
 'boats',
 'ext',
 None,
 'Ast',
 'Parks',
 'ester',
 '94',
 'relating',
 'vegetables',
 'accountable',
 'hyper',
 'Wim',
 'newest',
 'ENTITY/Rome',
 'ENTITY/Chancellor',
 'ENTITY/CBS',
 'businessman',
 'ENTITY/Delaware',
 'lands',
 'court',
 'aria',
 'approaching',
 'cker',
 'ENTITY/Salt',
 'Mak',
 'treating',
 'subsequently',
 'ENTITY/Ell',
 None,
 'ENTITY/180',
 'determination',
 'Salman',
 'Joel',
 'classified',
 'span',
 'earthquake',
 'ranked',
 '96',
 'ENTITY/Tiger',
 'advocacy',
 'mit',
 'colleges',
 'Yeah',
 'ENTITY/Captain',
 'orange',
 'projections',
 'electrical',
 'MA',
 'olog',
 'Newcastle',
 None,
 'representation',
 'lawsuits',
 'just',
 'aced',
 'Race',
 'Aqu',
 'Bills',
 'exclusively

In [14]:
mapping_train_data = []
for i, word in enumerate(common_vocab):
    
    if not word:
        continue
    
    if "ENTITY/" in word:
        w2v_emb = entities[word]
    else:
        w2v_emb = wordpiece[word.lower()]
        
    bert_emb = torch.squeeze(torch.squeeze(i_embeddings[i], dim = 0),dim = 0).detach().numpy()
    print("bert norm", np.linalg.norm(bert_emb))
    print("w2v_emb norm", np.linalg.norm(w2v_emb))
    #bert_emb = bert_emb/np.linalg.norm(bert_emb)
    #w2v_emb = w2v_emb/np.linalg.norm(w2v_emb)
    print(word)
    data_point={'word':word, 'wikipedia2vec':w2v_emb.tolist(), 'bert': bert_emb.tolist()}
    mapping_train_data.append(data_point)

len(mapping_train_data)

bert norm 2.295584
w2v_emb norm 2.3358235
the
bert norm 2.4234285
w2v_emb norm 2.8124087
to
bert norm 2.458268
w2v_emb norm 2.160507
and
bert norm 2.373503
w2v_emb norm 2.644827
of
bert norm 2.432008
w2v_emb norm 2.7094784
a
bert norm 2.3928869
w2v_emb norm 2.5782495
in
bert norm 2.44866
w2v_emb norm 2.9037478
for
bert norm 2.368081
w2v_emb norm 2.7709205
that
bert norm 2.4771185
w2v_emb norm 3.0100698
on
bert norm 2.3171222
w2v_emb norm 3.0193682
is
bert norm 2.438728
w2v_emb norm 2.7848876
with
bert norm 2.246134
w2v_emb norm 8.275168
ENTITY/The
bert norm 2.3470695
w2v_emb norm 2.935463
was
bert norm 2.529669
w2v_emb norm 3.308615
at
bert norm 2.4053588
w2v_emb norm 2.8312457
it
bert norm 2.5120802
w2v_emb norm 2.811539
as
bert norm 2.5908916
w2v_emb norm 3.42975
said
bert norm 2.7013094
w2v_emb norm 4.2591386
Ļ
bert norm 2.5852978
w2v_emb norm 3.3160017
be
bert norm 2.9197552
w2v_emb norm 3.9846184
s
bert norm 2.5999954
w2v_emb norm 3.0072646
by
bert norm 2.590858
w2v_emb norm 3.015

w2v_emb norm 6.0762596
ENTITY/25
bert norm 3.4294758
w2v_emb norm 4.5914326
land
bert norm 3.2297616
w2v_emb norm 3.290288
example
bert norm 2.9957275
w2v_emb norm 4.250212
authorities
bert norm 3.3550773
w2v_emb norm 4.1216173
date
bert norm 2.903838
w2v_emb norm 3.93676
ended
bert norm 3.4179976
w2v_emb norm 3.258799
all
bert norm 3.3974042
w2v_emb norm 8.680755
ENTITY/Reuters
bert norm 2.891892
w2v_emb norm 4.585013
businesses
bert norm 3.430973
w2v_emb norm 5.292891
ans
bert norm 3.1072018
w2v_emb norm 4.281964
details
bert norm 3.5090587
w2v_emb norm 4.5585527
ground
bert norm 2.9130442
w2v_emb norm 4.335264
pretty
bert norm 3.1980975
w2v_emb norm 3.3882682
ENTITY/Apple
bert norm 3.1604936
w2v_emb norm 5.4046965
ation
bert norm 2.9943008
w2v_emb norm 3.7822723
Smith
bert norm 3.2221107
w2v_emb norm 10.111975
ENTITY/Company
bert norm 2.9200857
w2v_emb norm 7.794873
ENTITY/Florida
bert norm 3.2180526
w2v_emb norm 5.1413093
drug
bert norm 3.1103508
w2v_emb norm 3.9193423
response
ber

bert norm 3.3143291
w2v_emb norm 4.8546414
Ms
bert norm 3.4692912
w2v_emb norm 4.353424
word
bert norm 3.5070336
w2v_emb norm 3.5300364
over
bert norm 2.9841309
w2v_emb norm 4.6817555
brother
bert norm 3.0067732
w2v_emb norm 4.0264635
necessary
bert norm 2.8216634
w2v_emb norm 3.3199003
eventually
bert norm 3.3115864
w2v_emb norm 10.171069
ENTITY/Star
bert norm 3.1449044
w2v_emb norm 4.5027895
send
bert norm 3.0361006
w2v_emb norm 4.668484
boy
bert norm 3.5714757
w2v_emb norm 5.3807216
Rs
bert norm 3.1788907
w2v_emb norm 4.1250362
remember
bert norm 2.785026
w2v_emb norm 3.4030986
21
bert norm 3.332037
w2v_emb norm 5.6939135
climate
bert norm 3.4435596
w2v_emb norm 4.8344
capacity
bert norm 3.4100208
w2v_emb norm 4.6096225
responsible
bert norm 3.1882706
w2v_emb norm 4.6089993
Matt
bert norm 3.1428287
w2v_emb norm 4.31078
month
bert norm 3.0436125
w2v_emb norm 4.8499246
suffered
bert norm 3.5821562
w2v_emb norm 6.1140237
og
bert norm 3.075008
w2v_emb norm 4.2434807
Peter
bert norm 3.20

bert norm 3.176454
w2v_emb norm 7.538668
ENTITY/Scotland
bert norm 3.7308326
w2v_emb norm 5.0603123
round
bert norm 3.8411267
w2v_emb norm 5.6594124
ith
bert norm 3.3166041
w2v_emb norm 4.4713545
breaking
bert norm 3.4143505
w2v_emb norm 5.8420396
voting
bert norm 3.1832745
w2v_emb norm 4.7515655
producer
bert norm 3.2126741
w2v_emb norm 11.052486
ENTITY/Love
bert norm 3.027877
w2v_emb norm 4.50513
remove
bert norm 3.6056376
w2v_emb norm 5.8436317
PA
bert norm 3.394309
w2v_emb norm 4.9609904
asset
bert norm 2.9534483
w2v_emb norm 4.437573
requires
bert norm 3.461178
w2v_emb norm 4.8853865
signing
bert norm 3.7976816
w2v_emb norm 5.1918936
ages
bert norm 2.7878768
w2v_emb norm 4.356736
impressive
bert norm 3.2835624
w2v_emb norm 4.896492
Irish
bert norm 3.3320696
w2v_emb norm 4.92522
authority
bert norm 3.3365135
w2v_emb norm 5.050126
ruled
bert norm 3.200198
w2v_emb norm 4.4624386
aimed
bert norm 3.1689875
w2v_emb norm 4.5389647
captain
bert norm 3.6633332
w2v_emb norm 5.521377
AG
bert

bert norm 3.4539208
w2v_emb norm 11.303939
ENTITY/Memorial
bert norm 3.5267358
w2v_emb norm 5.0934043
wave
bert norm 3.1765416
w2v_emb norm 4.4189963
fears
bert norm 3.0898993
w2v_emb norm 4.682375
kid
bert norm 3.1972117
w2v_emb norm 5.2691474
Giants
bert norm 3.2174737
w2v_emb norm 4.507072
recovered
bert norm 3.9486482
w2v_emb norm 5.230972
row
bert norm 3.495981
w2v_emb norm 10.169835
ENTITY/Radio
bert norm 3.1965997
w2v_emb norm 8.204132
ENTITY/Barcelona
bert norm 2.6822906
w2v_emb norm 4.3624296
wonderful
bert norm 3.71298
w2v_emb norm 5.2737474
Dow
bert norm 3.536685
w2v_emb norm 4.884449
stream
bert norm 3.328083
w2v_emb norm 4.454031
Simon
bert norm 3.4247267
w2v_emb norm 4.2413945
detail
bert norm 3.2741938
w2v_emb norm 4.4592385
volunteers
bert norm 3.5723298
w2v_emb norm 6.5623784
Ind
bert norm 3.3501508
w2v_emb norm 4.3774586
forms
bert norm 3.551913
w2v_emb norm 4.544881
mann
bert norm 3.3548763
w2v_emb norm 4.8659344
Ray
bert norm 3.8120244
w2v_emb norm 6.860204
oor
bert

...

bert norm 3.8790843
w2v_emb norm 5.5442963
655
bert norm 3.9229536
w2v_emb norm 4.7545185
Conversion
bert norm 3.8840618
w2v_emb norm 5.875844
mL
bert norm 3.921146
w2v_emb norm 11.320548
ENTITY/Border
bert norm 3.7721298
w2v_emb norm 6.6544747
ENTITY/Ë
bert norm 3.945993
w2v_emb norm 5.4267526
Factor
bert norm 3.536741
w2v_emb norm 11.049405
ENTITY/Number
bert norm 3.687183
w2v_emb norm 2.4245598
ejac
bert norm 3.8746474
w2v_emb norm 5.614219
Cho
bert norm 3.7531042
w2v_emb norm 5.3889093
righteousness
bert norm 4.0394363
w2v_emb norm 4.486887
PATH
bert norm 3.8800035
w2v_emb norm 3.9434075
Elys
bert norm 3.9378026
w2v_emb norm 5.4608192
faculties
bert norm 3.778393
w2v_emb norm 10.874105
ENTITY/Earthquake
bert norm 3.8663201
w2v_emb norm 4.184355
References
bert norm 3.935175
w2v_emb norm 5.8514457
buff
bert norm 3.6170866
w2v_emb norm 13.909046
ENTITY/1895
bert norm 3.7571752
w2v_emb norm 6.325309
colo
bert norm 4.061725
w2v_emb norm 8.862666
ENTITY/Vi
bert norm 3.9029465
w2v_emb no

bert norm 3.8311903
w2v_emb norm 5.9206862
Gors
bert norm 3.815516
w2v_emb norm 11.720044
ENTITY/Duel
bert norm 3.923306
w2v_emb norm 5.4027314
admins
bert norm 3.9394686
w2v_emb norm 6.948442
ENTITY/Flor
bert norm 3.9410887
w2v_emb norm 9.43142
ENTITY/Deus
bert norm 4.130965
w2v_emb norm 6.2160206
cham
bert norm 3.7506492
w2v_emb norm 5.0317516
Rails
bert norm 3.9303875
w2v_emb norm 5.4977055
ceptor
bert norm 4.1352997
w2v_emb norm 3.4696736
naire
bert norm 3.9739335
w2v_emb norm 9.747762
ENTITY/Squid
bert norm 3.899261
w2v_emb norm 10.352557
ENTITY/Warranty
bert norm 3.7844043
w2v_emb norm 5.493911
SPEC
bert norm 3.9679778
w2v_emb norm 6.5837884
ensis
bert norm 4.07436
w2v_emb norm 4.424245
FUN
bert norm 3.7491865
w2v_emb norm 5.1682334
stellar
bert norm 3.8248994
w2v_emb norm 4.372141
Select
bert norm 3.8815868
w2v_emb norm 3.3677714
arget
bert norm 3.7910893
w2v_emb norm 10.012594
ENTITY/Uncharted
bert norm 3.754923
w2v_emb norm 4.281964
Details
bert norm 3.778931
w2v_emb norm 6.03

bert norm 3.8062665
w2v_emb norm 4.499876
arthy
bert norm 3.9228878
w2v_emb norm 10.211814
ENTITY/Psychic
bert norm 3.8006036
w2v_emb norm 7.015316
dorsal
bert norm 3.678271
w2v_emb norm 2.3675685
cember
bert norm 3.6922946
w2v_emb norm 6.2371497
joice
bert norm 3.909689
w2v_emb norm 3.849451
uint
bert norm 3.8510156
w2v_emb norm 3.9718673
derog
bert norm 4.142574
w2v_emb norm 3.8294382
Subject
bert norm 3.9405663
w2v_emb norm 3.7278402
hemat
bert norm 3.9922662
w2v_emb norm 5.6332207
meshes
bert norm 3.8269312
w2v_emb norm 5.7795725
Terran
bert norm 4.0787387
w2v_emb norm 5.040955
Load
bert norm 3.6705363
w2v_emb norm 5.2660456
goblins
bert norm 3.918046
w2v_emb norm 4.605116
Shattered
bert norm 3.8831456
w2v_emb norm 4.9881554
tests
bert norm 4.115881
w2v_emb norm 5.0853877
Spread
bert norm 3.7783196
w2v_emb norm 9.015059
ENTITY/Naruto
bert norm 3.5713556
w2v_emb norm 2.5441256
predic
bert norm 3.9297967
w2v_emb norm 5.420382
Hyp
bert norm 3.9164755
w2v_emb norm 8.390921
ENTITY/Arkha

bert norm 3.6811836
w2v_emb norm 6.8010654
integer
bert norm 3.8121836
w2v_emb norm 6.2708287
regex
bert norm 3.0098765
w2v_emb norm 6.5504246
nomine
bert norm 3.1529586
w2v_emb norm 5.777616
subparagraph
bert norm 4.0332465
w2v_emb norm 5.9146333
Header
bert norm 3.8797193
w2v_emb norm 5.241732
Spawn
bert norm 3.8609357
w2v_emb norm 5.9294143
toggle
bert norm 3.641218
w2v_emb norm 4.974119
Abyss
bert norm 3.5815713
w2v_emb norm 6.1967025
expr
bert norm 3.80759
w2v_emb norm 6.4719214
Zerg
bert norm 3.6652973
w2v_emb norm 9.214765
ENTITY/Grimoire
bert norm 4.194874
w2v_emb norm 4.532595
Contents
bert norm 3.762523
w2v_emb norm 3.0809548
Instance
bert norm 3.324495
w2v_emb norm 8.049925
cyclopedia
bert norm 3.3646529
w2v_emb norm 5.5112553
Takeru
bert norm 3.2571828
w2v_emb norm 6.764012
rgb
bert norm 3.1371999
w2v_emb norm 6.4929523
htt
bert norm 2.679919
w2v_emb norm 4.8394094
bryce
bert norm 2.9951315
w2v_emb norm 3.593342
livest
bert norm 3.3725355
w2v_emb norm 5.4912477
Annotations


45451

In [15]:
filename = 'data/mapping_train_data_entities_roberta.jsonl'
with open(filename, "w", encoding='utf-8') as fp:
        for row in mapping_train_data:
            fp.write(json.dumps(row) + "\n")

In [56]:
6.3212e-01

0.63212

In [63]:
2.3254e-01

0.23254

In [65]:
8.5548e-02

0.085548

In [66]:
5.7642e-04

0.00057642

In [67]:
one = torch.nn.Softmax()(torch.flip(torch.arange(1, 1000, 1), dims=[0]).float())

/tmp/ipykernel_2974810/2000997586.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  one = torch.nn.Softmax()(torch.flip(torch.arange(1, 1000, 1), dims=[0]).float())


In [69]:
one.unsqueeze(dim=0).expand(128, -1).shape

torch.Size([128, 999])

In [81]:
with open("./aggregations.json") as f:
    aggregations = json.load(f)

In [82]:
for s in aggregations:
    for rel in aggregations[s]:
        if len(aggregations[s][rel]) == 1:
            continue
        
        ids = tokenizer.convert_tokens_to_ids(aggregations[s][rel])
        bert_average = torch.mean(torch.stack([i_embeddings[i] for i in ids], dim=1), dim=1)
        w2v_average = torch.mean(torch.stack([torch.tensor(entities[common_vocab[i]]) if common_vocab[i] in entities else torch.tensor(wordpiece[common_vocab[i].lower()]) for i in ids], dim=1), dim=1)
        data_point={'word':", ".join(aggregations[s][rel]), 'wikipedia2vec':w2v_average, 'bert': bert_average}
        mapping_train_data.append(data_point)

len(mapping_train_data)

23178

In [83]:
[w['word'] for w in mapping_train_data[22143:]]

['Syria, Bulgaria, Armenia',
 'Russia, Brazil, Australia, Germany, Mongolia, Canada, Greece, Italy',
 'Zambia, Angola',
 'Australia, Germany, Canada',
 'NATO, UNESCO',
 'Bavaria, Prussia',
 'Czechoslovakia, Switzerland, Poland, Luxembourg, Belgium',
 'Mali, Singapore, Cuba, Norway, Paraguay, Angola, Thailand, Azerbaijan, Botswana, Guyana, Peru, Cameroon, Jordan, Kenya, Australia, Ghana, Belgium, Bulgaria, Uganda, Iran, Algeria, Zambia, Hungary, Ireland, Turkey, Sweden, Uzbekistan, Benin, Serbia, Belarus, Bangladesh, Switzerland, Honduras, Indonesia, Israel, Madagascar, Somalia, Panama, Bhutan, Poland, Guatemala, Iraq, Colombia, Japan, Ukraine, Tunisia, Cyprus, Uruguay, Gabon, Rwanda, Slovenia, Mozambique, Ethiopia, Greece, Chad, Belize, Denmark, Chile, Brazil, Mongolia, Kazakhstan, Argentina, Mexico, Portugal, Finland, Bolivia, Italy, Albania, Moldova, Venezuela, Namibia, Senegal, Spain, Austria, Malta, Netherlands, Croatia, Luxembourg, Zimbabwe, Monaco, Egypt, Libya',
 'Europe, NATO',

In [84]:
filename = 'data/mapping_train_data_entites_words_averages.jsonl'
with open(filename, "w", encoding='utf-8') as fp:
        for row in mapping_train_data:
            fp.write(json.dumps(row) + "\n")

TypeError: Object of type Tensor is not JSON serializable